In [1]:
import pandas as pd
import regex as re
import numpy as np
from datetime import datetime as dt
from datetime import timedelta

In [2]:
path = 'C:/Users/glh19/OneDrive/Desktop/Winter_Spring 2022/Tosun-Turgut/data/'

adni1 = pd.read_csv(path + 'adni1_muse_results_with_dates.csv')
adni2 = pd.read_csv(path + 'adni2_muse_results_with_dates.csv')
adni3 = pd.read_csv(path + 'adni3_muse_results_with_dates.csv')

DATADIC = pd.read_csv(path + 'DATADIC.csv')
DXSUM = pd.read_csv(path + 'DXSUM_PDXCONV_ADNIALL.csv')
PTDEMOG = pd.read_csv(path + 'PTDEMOG.csv')

### ADNI Cleaning

In [3]:
adni1.head(5)
# necessary and/or useful columns to do future calculations on:

# icv value and % icv value: code, roi_volume, roi_name

# examDate needed for age calculation, 
# examDate original format: %m/%day/%Y
# need to format to datetime: %Y-%m-%d

# volumes must be cleared of unnecessary values as specified below

# icv will have to be removed later in cleaning, post merge

# examDate column must be filled of missing values


,project,code,roi_idx,roi_volume,roi_name,roi_index,examDate
0,ADNI1,002_S_0295X1a_118671,NaN,1.266543e+06,TOTALBRAIN,701.0,4/18/2006
1,ADNI1,002_S_0295X1a_118671,NaN,5.994261e+05,GM,601.0,4/18/2006
2,ADNI1,002_S_0295X1a_118671,NaN,6.019957e+05,WM,604.0,4/18/2006
3,ADNI1,002_S_0295X1a_118671,NaN,2.929851e+05,GM_L,606.0,4/18/2006
4,ADNI1,002_S_0295X1a_118671,NaN,2.935244e+05,WM_L,607.0,4/18/2006


In [4]:
# Normalize the format of 'examDate' column for each dataframe
# desired format: %Y-%m-%d

# accomplishing clearning individual to maintain integrity of indices
adni3['examDate'] = pd.to_datetime(adni3['examDate']).dt.date
adni1['examDate'] = pd.to_datetime(adni1['examDate']).dt.date
adni2['examDate'] = pd.to_datetime(adni2['examDate']).dt.date

# adni1['examDate']
# adni2['examDate']
# adni3['examDate']

#icv value column for each dataframe:
adni1['icv_values'] = (adni1[adni1['roi_name'] == 'icv']['roi_volume'])
adni2['icv_values'] = (adni2[adni2['roi_name'] == 'icv']['roi_volume'])
adni3['icv_values'] = (adni3[adni3['roi_name'] == 'icv']['roi_volume'])
# adni3['thing'] at this point will have Nas

#fillna(0) for non-icv values:may change column name to 'icv_vals'
adni1['icv_values'] = adni1['icv_values'].fillna(0)
adni2['icv_values'] = adni2['icv_values'].fillna(0)
adni3['icv_values'] = adni3['icv_values'].fillna(0)
# adni3['thing'] 0s for all values other than icv values such as 1652002.0

In [5]:
# desired grouping when calculting %icv for each dataframe
# adni1.groupby('code')['thing'].max()

# function to divide roi_volume by icv values 
def divide_two_cols(adni):
    adni['%_icv'] = adni['roi_volume'] / adni['icv_values'].max()
    return adni['%_icv']

adni1['%_icv'] = adni1.groupby('code').apply(divide_two_cols).tolist()
adni2['%_icv'] = adni2.groupby('code').apply(divide_two_cols).tolist()
adni3['%_icv'] = adni3.groupby('code').apply(divide_two_cols).tolist()

# check correct % icv calculations:
adni3[((adni3['roi_name']=='icv') | (adni3['roi_name']== 'Left Amygdala'))][['project','code','roi_name','roi_volume','icv_values','%_icv']]


,project,code,roi_name,roi_volume,icv_values,%_icv
118,ADNI3,ADNI3_002_S_0413y00_i863056,Left Amygdala,1.058531e+03,0.000000e+00,0.000582
258,ADNI3,ADNI3_002_S_0413y00_i863056,icv,1.819225e+06,1.819225e+06,1.000000
377,ADNI3,ADNI3_002_S_0413y02_i1221051,Left Amygdala,8.900000e+02,0.000000e+00,0.000624
517,ADNI3,ADNI3_002_S_0413y02_i1221051,icv,1.425667e+06,1.425667e+06,1.000000
636,ADNI3,ADNI3_002_S_1155y00_i843510,Left Amygdala,1.094572e+03,0.000000e+00,0.000662
...,...,...,...,...,...,...
359165,ADNI3,ADNI3_941_S_6581y02_i1343754,icv,1.713419e+06,1.713419e+06,1.000000
359172,ADNI3,ADNI3_941_S_6854y00_i1291638,Left Amygdala,9.040000e+02,0.000000e+00,0.000551
359318,ADNI3,ADNI3_941_S_6854y00_i1291638,icv,1.641144e+06,1.641144e+06,1.000000
359325,ADNI3,ADNI3_941_S_6854y01_i1428390,Left Amygdala,8.070000e+02,0.000000e+00,0.000488


In [6]:
# merge adni dataframes, remove unwanted regions, split code column

# merge:
adni12 = pd.concat([adni1,adni2], axis=0)
adni123 = pd.concat([adni12, adni3], axis=0)
# adni123

# discard unwanted brain regions:

# to remove: uppercase items
adni123.drop(adni123.index[adni123['roi_name'].str.isupper()], inplace=True)

discard = (['Right Cerebellum Exterior', 'Left Cerebellum Exterior',
            'Right Cerebellum White Matter','Left Cerebellum White Matter',
            'frontal lobe WM right','frontal lobe WM left',
            'occipital lobe WM right','occipital lobe WM left',
            'parietal lobe WM right','parietal lobe WM left',
            'temporal lobe WM right','temporal lobe WM left',
            'fornix right','fornix left','corpus callosum'])

adni123 = adni123[adni123.roi_name.isin(discard) == False]
# adni123

#split code:

#remove unwanted ADNI3 partial string from code entries:
bonus_pat = r'[ADNI]+\d+_'

adni123['code'] = adni123['code'].str.replace("[ADNI]+\d+_","")
adni123['split_code'] = adni123['code'].str.split('_')

# split up code into new columns: PTID, RID, unique image #
# example code: 002_S_0295xia_118671

adni123['SITEID'] = adni123['split_code'].str[0]
adni123['RID_VISCODE'] = adni123['split_code'].str[2]
adni123['IMAGEUID'] = adni123['split_code'].str[3]

# RID=1326 and VISCODE=m12
rid_pattern3 = r'(\d\d\d\d)'
adni123['RID_VISCODE'] =  adni123['RID_VISCODE'].str.split(pat=rid_pattern3)

adni123['RID'] = adni123['RID_VISCODE'].str[1]
adni123['RID'] = adni123['RID'].str.lstrip('0')
adni123['VISCODE'] = adni123['RID_VISCODE'].str[2]

# adni123

In [7]:
# check if everything remains functional: 
# adni123
# adni123['examDate']
# adni123['RID']

In [8]:
adni123.columns

Index(['project', 'code', 'roi_idx', 'roi_volume', 'roi_name', 'roi_index',
       'examDate', 'icv_values', '%_icv', 'split_code', 'SITEID',
       'RID_VISCODE', 'IMAGEUID', 'RID', 'VISCODE'],
      dtype='object')

In [9]:
adni123.head(1)

,project,code,roi_idx,roi_volume,roi_name,roi_index,examDate,icv_values,%_icv,split_code,SITEID,RID_VISCODE,IMAGEUID,RID,VISCODE
113,ADNI1,002_S_0295X1a_118671,NaN,1673.583956,3rd Ventricle,4.0,2006-04-18,0.0,0.000921,"[002, S, 0295X1a, 118671]",002,"[, 0295, X1a]",118671,295,X1a


### DXSUM Cleaning  

In [10]:
DXSUM.head(5)
# necessary and/or useful columns to do future calculations on:

# Keep: Phase	ID	RID	PTID	SITEID
# Keep until end/export: rest

# * EXAMDATE for age calculation
# column must be filled of missing values
# * DXCHANGE, DXCURREN, and DIAGNOSIS for complete diagnosis column

# final cleaning: removing ADNIGO cases at end as to not interupt indices

# Note: PTID values in format: 022_S_0004

,Phase,ID,RID,PTID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,EXAMDATE,...,DXPATYP,DXPOTHSP,DXDEP,DXDEPSP,DXOTHDEM,DXODES,DXOOTHSP,DXCONFID,DIAGNOSIS,update_stamp
0,ADNI1,2.0,2,011_S_0002,107,bl,bl,2005-10-01,NaN,2005-09-29,...,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,4.0,NaN,2005-10-01 00:00:00.0
1,ADNI1,336.0,2,011_S_0002,107,m06,m06,2006-04-27,NaN,2006-03-06,...,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,3.0,NaN,2006-04-27 00:00:00.0
2,ADNI1,4.0,3,011_S_0003,107,bl,bl,2005-10-01,NaN,2005-09-30,...,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,3.0,NaN,2005-10-01 00:00:00.0
3,ADNI1,338.0,3,011_S_0003,107,m06,m06,2006-04-27,NaN,2006-03-13,...,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,4.0,NaN,2006-04-27 00:00:00.0
4,ADNI1,990.0,3,011_S_0003,107,m12,m12,2006-09-14,NaN,2006-09-13,...,-4.0,-4,NaN,NaN,-4.0,-4.0,-4,4.0,NaN,2006-09-14 00:00:00.0


In [11]:
# curate a complete diagnosis column for DXSUM:
DXSUM['CHAN_CURR'] = DXSUM['DXCHANGE'].fillna(value=DXSUM['DXCURREN'])
DXSUM['complete_diagnosis'] = DXSUM['CHAN_CURR'].fillna(value=DXSUM['DIAGNOSIS'])
DXSUM['complete_diagnosis']
# DXSUM.head(5)

0        1.0
1        1.0
2        3.0
3        3.0
4        3.0
        ... 
12889    1.0
12890    1.0
12891    1.0
12892    2.0
12893    1.0
Name: complete_diagnosis, Length: 12894, dtype: float64

### DATADIC Cleaning  

In [12]:
DATADIC.head(5)
# necessary and/or useful columns to do future calculations on:
# Phase: 'ADNI1', 'ADNIGO', 'ADNI2', 'ADNI3'

# may not be necessary to merge

,Phase,FLDNAME,TBLNAME,CRFNAME,TEXT,TYPE,LENGTH,CODE,UNITS
0,ADNI1,ID,AAL,NaN,Record ID,N,38,"""crfname"","""",""indexes"",""adni_aal_idx=TBLID,FLD...",NaN
1,ADNI1,SX,AAL,NaN,NaN,N,10,NaN,NaN
2,ADNI1,USERID,AAL,NaN,NaN,T,100,NaN,NaN
3,ADNI1,USERDATE,AAL,NaN,Date record created,S,0,NaN,NaN
4,ADNI1,USERID2,AAL,NaN,NaN,T,100,NaN,NaN


### PTDEMOG Cleaning  

In [13]:
PTDEMOG.head(5)
# necessary and/or useful columns to do future calculations on:

# Keep until end/export: 
# Phase	ID	RID	SITEID	VISCODE	

# Use for Calculation of age of individual: 
#* PTDOBMM, PTDOBYY 

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,PTSOURCE,PTGENDER,...,PTTLANG,PTPLANG,PTPSPEC,PTCOGBEG,PTMCIBEG,PTADBEG,PTADDX,PTETHCAT,PTRACCAT,update_stamp
0,ADNI1,18,2,107,sc,sc,2005-08-17,NaN,1,1.0,...,1.0,1.0,-4,NaN,NaN,-4.0,NaN,2.0,5.0,2005-08-17 00:00:00.0
1,ADNI1,20,1,10,f,f,2005-08-18,NaN,1,2.0,...,-4.0,-4.0,-4,NaN,NaN,-4.0,NaN,-4.0,-4.0,2005-08-18 00:00:00.0
2,ADNI1,22,3,107,sc,sc,2005-08-18,NaN,1,1.0,...,1.0,1.0,-4,NaN,NaN,1999.0,NaN,2.0,5.0,2005-08-18 00:00:00.0
3,ADNI1,24,4,10,sc,sc,2005-08-18,NaN,1,1.0,...,2.0,2.0,-4,NaN,NaN,-4.0,NaN,1.0,5.0,2005-08-18 00:00:00.0
4,ADNI1,26,5,107,sc,sc,2005-08-23,NaN,1,1.0,...,1.0,1.0,-4,NaN,NaN,-4.0,NaN,2.0,5.0,2005-08-23 00:00:00.0


In [14]:
PTDEMOG['PTDOBYY']

# Name: PTDOBYY, Length: 4768, dtype: float64

0       1931.0
1       1944.0
2       1924.0
3       1938.0
4       1931.0
         ...  
4763    1973.0
4764    2022.0
4765    1953.0
4766    1956.0
4767    1960.0
Name: PTDOBYY, Length: 4768, dtype: float64

# FULL MERGE

In [15]:
# Potential merges: 

# approach 1) adni123 merge with DXSUM on key value: RID
# combination: should include: both forms of examDate
# should check if two columns are equal
# then merge with PTDEMOG on 'RID'

# *Note: so far successful when running this approach, until reaching age calc

# adni_DXSUM_merged = pd.merge(adni123.astype(str), DXSUM.astype(str), on='RID')
# full_merged = pd.merge(adni_DXSUM_merged.astype(str), PTDEMOG.astype(str), on='RID')

#*******************************************************

# approach 2) PTDEMOG merge with DXSUM, then merge with adni123 
# proven issue: merging PTDEMOG and DXSUM leads to loss of 'DOB' col

# PTDEMOG_DXSUM = PTDEMOG.merge(DXSUM,  how='right')
# full_merged = pd.merge(adni123.astype(str), PTDEMOG_DXSUM.astype(str), on='RID')


#current working version ***
# PTDEMOG_DXSUM = pd.merge(PTDEMOG.astype(str), DXSUM.astype(str), on='RID')
# full_merged = pd.merge(adni123.astype(str), PTDEMOG_DXSUM.astype(str), on='RID')
# end current working version ***

PTDEMOG_DXSUM = pd.merge(PTDEMOG.astype(str), DXSUM.astype(str), on='RID')
full_merged = pd.merge(adni123.astype(str), PTDEMOG_DXSUM.astype(str), on='RID')

# *Note: so far successful when running this approach, until reaching age calc

#*******************************************************

# approach 3) adni123 merge with PTDEMOG on key value: RID
# then merge with DXSUM
# *Note: so far successful when running this approach, until reaching age calc

# adni_PTDEMOG_merged = pd.merge(adni123.astype(str), PTDEMOG.astype(str), on='RID')
# full_merged = pd.merge(adni_PTDEMOG_merged.astype(str), DXSUM.astype(str), on='RID')


# full_merged

In [16]:
# full_merged['PTDOBYY']
# pd.to_datetime(full_merged['PTDOBYY']).dt.year

# PTDOBYY/MM as DATETIME

In [17]:
# dob_year = pd.to_datetime(full_merged['PTDOBYY']).dt.year
full_merged['PTDOBYY'] = pd.to_datetime(full_merged['PTDOBYY'],format='%Y.0').dt.year

# pd.to_datetime(full_merged['DOB']).dt.year
# pd.to_datetime(full_merged['DOB']).dt.month
# pd.to_datetime(full_merged['DOB']).dt.day

In [18]:
#  dob_month = pd.to_datetime(full_merged['PTDOBMM'], format='%m').dt.month

full_merged['PTDOBMM'] = pd.to_datetime(full_merged['PTDOBMM'], format='%m.0').dt.month


# RENAME PTDOBMM/YY as MONTH/YEAR for DATETIME

In [19]:
# must rename: full_merged['PTDOBYY'], full_merged['PTDOBMM']

full_merged.rename(columns={'PTDOBYY':'YEAR','PTDOBMM':'MONTH'}, inplace=True)
full_merged.columns

Index(['project', 'code', 'roi_idx', 'roi_volume', 'roi_name', 'roi_index',
       'examDate', 'icv_values', '%_icv', 'split_code',
       ...
       'DXDEP', 'DXDEPSP', 'DXOTHDEM', 'DXODES', 'DXOOTHSP', 'DXCONFID',
       'DIAGNOSIS', 'update_stamp_y', 'CHAN_CURR', 'complete_diagnosis'],
      dtype='object', length=101)

In [21]:
# full_merged['YEAR']

# full_merged['MONTH']
full_merged['PTGENDER']

0          1.0
1          1.0
2          1.0
3          1.0
4          1.0
          ... 
8110574    1.0
8110575    1.0
8110576    1.0
8110577    1.0
8110578    1.0
Name: PTGENDER, Length: 8110579, dtype: object

### Revive unique image identifier column

In [22]:
# # split up code into new columns: PTID, RID, unique image #
# # example code: 002_S_0295xia_118671

full_merged['IMAGEUID'] = full_merged['code'].str.split('_').str[3]
full_merged['IMAGEUID'] = full_merged['IMAGEUID'].str.replace('i','')


In [23]:
full_merged['IMAGEUID']

0           118671
1           118671
2           118671
3           118671
4           118671
            ...   
8110574    1428390
8110575    1428390
8110576    1428390
8110577    1428390
8110578    1428390
Name: IMAGEUID, Length: 8110579, dtype: object

# JOIN YEAR and MONTH col as DOB

In [24]:
full_merged['DOB_DATE'] = pd.to_datetime(full_merged[['YEAR', 'MONTH']].assign(DAY=1))

full_merged['DOB_DATE']

0         1921-06-01
1         1921-06-01
2         1921-06-01
3         1921-06-01
4         1921-06-01
             ...    
8110574   1933-11-01
8110575   1933-11-01
8110576   1933-11-01
8110577   1933-11-01
8110578   1933-11-01
Name: DOB_DATE, Length: 8110579, dtype: datetime64[ns]

In [25]:
full_merged['examDate'] 
# Name: examDate, Length: 8110579, dtype: object

# full_merged['DOB']
# Name: DOB, Length: 8110579, dtype: object

0          2006-04-18
1          2006-04-18
2          2006-04-18
3          2006-04-18
4          2006-04-18
              ...    
8110574    2021-04-05
8110575    2021-04-05
8110576    2021-04-05
8110577    2021-04-05
8110578    2021-04-05
Name: examDate, Length: 8110579, dtype: object

In [26]:
# compute age:

# approach 1:calculate mid merge of two dataframes

# adni_merged['age_in_days'] = adni_merged['examDate'] - adni_merged['DOB']
# adni_merged['age_in_years'] = adni_merged["age_in_days"] / timedelta(days=365)

# approach 2:calculate post all merges

#format string cols into datetime again:
full_merged['examDate']  = pd.to_datetime(full_merged['examDate'], format='%Y-%m-%d')
full_merged['DOB_DATE'] = pd.to_datetime(full_merged['DOB_DATE'], format='%Y-%m-%d')
# however, when attempting to do this:
# ValueError: time data nan-nan-15 doesn't match format specified

full_merged['age_in_days'] = full_merged['examDate'] - full_merged['DOB_DATE']
full_merged['age_in_years'] = full_merged["age_in_days"] / timedelta(days=365)

# full_merged
full_merged['age_in_years']


# current error for all three merge options: TypeError: unsupported operand type(s) for -: 'str' and 'str'
# idea: change string columns for date back into datetime


0          84.936986
1          84.936986
2          84.936986
3          84.936986
4          84.936986
             ...    
8110574    87.484932
8110575    87.484932
8110576    87.484932
8110577    87.484932
8110578    87.484932
Name: age_in_years, Length: 8110579, dtype: float64

# Next Steps

In [27]:
# dropping ADNIGO
# icv will have to be removed later in cleaning, post merge

In [28]:
# Drop ADNIGO rows

full_merged = full_merged[full_merged['project'] != 'ADNIGO']

full_merged.head(5)

,project,code,roi_idx,roi_volume,roi_name,roi_index,examDate,icv_values,%_icv,split_code,...,DXODES,DXOOTHSP,DXCONFID,DIAGNOSIS,update_stamp_y,CHAN_CURR,complete_diagnosis,DOB_DATE,age_in_days,age_in_years
0,ADNI1,002_S_0295X1a_118671,nan,1673.5839558839798,3rd Ventricle,4.0,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",...,-4.0,-4,4.0,nan,2006-05-10 00:00:00.0,1.0,1.0,1921-06-01,31002 days,84.936986
1,ADNI1,002_S_0295X1a_118671,nan,1673.5839558839798,3rd Ventricle,4.0,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",...,-4.0,-4,4.0,nan,2006-11-10 00:00:00.0,1.0,1.0,1921-06-01,31002 days,84.936986
2,ADNI1,002_S_0295X1a_118671,nan,1673.5839558839798,3rd Ventricle,4.0,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",...,-4.0,-4,4.0,nan,2007-05-29 00:00:00.0,1.0,1.0,1921-06-01,31002 days,84.936986
3,ADNI1,002_S_0295X1a_118671,nan,1673.5839558839798,3rd Ventricle,4.0,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",...,-4.0,-4,4.0,nan,2008-09-05 00:00:00.0,1.0,1.0,1921-06-01,31002 days,84.936986
4,ADNI1,002_S_0295X1a_118671,nan,1673.5839558839798,3rd Ventricle,4.0,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",...,-4.0,-4,4.0,nan,2009-05-26 00:00:00.0,1.0,1.0,1921-06-01,31002 days,84.936986


In [29]:
# Drop icv volumne rows

# this didn't seem to work...
# discard_icv = (['icv'])
# full_merged = full_merged[full_merged.roi_name.isin(discard_icv) == False]

# this worked 
full_merged = full_merged[full_merged['roi_name'] != 'icv']

full_merged.head(5)

,project,code,roi_idx,roi_volume,roi_name,roi_index,examDate,icv_values,%_icv,split_code,...,DXODES,DXOOTHSP,DXCONFID,DIAGNOSIS,update_stamp_y,CHAN_CURR,complete_diagnosis,DOB_DATE,age_in_days,age_in_years
0,ADNI1,002_S_0295X1a_118671,nan,1673.5839558839798,3rd Ventricle,4.0,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",...,-4.0,-4,4.0,nan,2006-05-10 00:00:00.0,1.0,1.0,1921-06-01,31002 days,84.936986
1,ADNI1,002_S_0295X1a_118671,nan,1673.5839558839798,3rd Ventricle,4.0,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",...,-4.0,-4,4.0,nan,2006-11-10 00:00:00.0,1.0,1.0,1921-06-01,31002 days,84.936986
2,ADNI1,002_S_0295X1a_118671,nan,1673.5839558839798,3rd Ventricle,4.0,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",...,-4.0,-4,4.0,nan,2007-05-29 00:00:00.0,1.0,1.0,1921-06-01,31002 days,84.936986
3,ADNI1,002_S_0295X1a_118671,nan,1673.5839558839798,3rd Ventricle,4.0,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",...,-4.0,-4,4.0,nan,2008-09-05 00:00:00.0,1.0,1.0,1921-06-01,31002 days,84.936986
4,ADNI1,002_S_0295X1a_118671,nan,1673.5839558839798,3rd Ventricle,4.0,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",...,-4.0,-4,4.0,nan,2009-05-26 00:00:00.0,1.0,1.0,1921-06-01,31002 days,84.936986


In [30]:
full_merged[full_merged['roi_name'] == 'icv'].head(5)

,project,code,roi_idx,roi_volume,roi_name,roi_index,examDate,icv_values,%_icv,split_code,...,DXODES,DXOOTHSP,DXCONFID,DIAGNOSIS,update_stamp_y,CHAN_CURR,complete_diagnosis,DOB_DATE,age_in_days,age_in_years


In [31]:
# First check which columns have NaN values:
full_merged[full_merged.columns[full_merged.isna().any()]]

# Drop rows with missing data in the following columns:
# examDate	MONTH	YEAR	DOB_DATE	age_in_days	age_in_years

,examDate,MONTH,YEAR,DOB_DATE,age_in_days,age_in_years
0,2006-04-18,6.0,1921.0,1921-06-01,31002 days,84.936986
1,2006-04-18,6.0,1921.0,1921-06-01,31002 days,84.936986
2,2006-04-18,6.0,1921.0,1921-06-01,31002 days,84.936986
3,2006-04-18,6.0,1921.0,1921-06-01,31002 days,84.936986
4,2006-04-18,6.0,1921.0,1921-06-01,31002 days,84.936986
...,...,...,...,...,...,...
8110571,2021-04-05,11.0,1933.0,1933-11-01,31932 days,87.484932
8110572,2021-04-05,11.0,1933.0,1933-11-01,31932 days,87.484932
8110573,2021-04-05,11.0,1933.0,1933-11-01,31932 days,87.484932
8110574,2021-04-05,11.0,1933.0,1933-11-01,31932 days,87.484932


In [32]:
print(full_merged.columns)

Index(['project', 'code', 'roi_idx', 'roi_volume', 'roi_name', 'roi_index',
       'examDate', 'icv_values', '%_icv', 'split_code',
       ...
       'DXODES', 'DXOOTHSP', 'DXCONFID', 'DIAGNOSIS', 'update_stamp_y',
       'CHAN_CURR', 'complete_diagnosis', 'DOB_DATE', 'age_in_days',
       'age_in_years'],
      dtype='object', length=104)


In [33]:
# full_merged = (full_merged[['project', 'code', 'roi_volume',
#         'roi_name', 'roi_index','examDate', 'icv_values', '%_icv',
#         'split_code','complete_diagnosis', 'DOB_DATE', 
#         'age_in_days','age_in_years', 'IMAGEUID']])

full_merged = (full_merged[['project', 'code', 'roi_volume',
        'roi_name', 'examDate', 'icv_values', '%_icv',
        'split_code','complete_diagnosis', 'PTGENDER', 'DOB_DATE', 
        'age_in_years', 'IMAGEUID']])

full_merged.head(5)

,project,code,roi_volume,roi_name,examDate,icv_values,%_icv,split_code,complete_diagnosis,PTGENDER,DOB_DATE,age_in_years,IMAGEUID
0,ADNI1,002_S_0295X1a_118671,1673.5839558839798,3rd Ventricle,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",1.0,1.0,1921-06-01,84.936986,118671
1,ADNI1,002_S_0295X1a_118671,1673.5839558839798,3rd Ventricle,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",1.0,1.0,1921-06-01,84.936986,118671
2,ADNI1,002_S_0295X1a_118671,1673.5839558839798,3rd Ventricle,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",1.0,1.0,1921-06-01,84.936986,118671
3,ADNI1,002_S_0295X1a_118671,1673.5839558839798,3rd Ventricle,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",1.0,1.0,1921-06-01,84.936986,118671
4,ADNI1,002_S_0295X1a_118671,1673.5839558839798,3rd Ventricle,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",1.0,1.0,1921-06-01,84.936986,118671


In [34]:
# the method will drop all rows with at least one missing value.
full_merged = full_merged.dropna()

# Note: If I want to maintain the entire dataframe curated above, 
# but want to drop rows with NaN values present in only a particular
# set of columns:
# df = df.dropna(subset=['colA', 'colC'])

In [35]:
# full_merged.head(5)

full_merged[full_merged.columns[full_merged.isna().any()]]
# the above .dropna appears to have been successful when checking for 
# remaining columns with NaN values/ 

""
0
1
2
3
4
...
8110571
8110572
8110573
8110574


In [36]:
full_merged.head(5)

,project,code,roi_volume,roi_name,examDate,icv_values,%_icv,split_code,complete_diagnosis,PTGENDER,DOB_DATE,age_in_years,IMAGEUID
0,ADNI1,002_S_0295X1a_118671,1673.5839558839798,3rd Ventricle,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",1.0,1.0,1921-06-01,84.936986,118671
1,ADNI1,002_S_0295X1a_118671,1673.5839558839798,3rd Ventricle,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",1.0,1.0,1921-06-01,84.936986,118671
2,ADNI1,002_S_0295X1a_118671,1673.5839558839798,3rd Ventricle,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",1.0,1.0,1921-06-01,84.936986,118671
3,ADNI1,002_S_0295X1a_118671,1673.5839558839798,3rd Ventricle,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",1.0,1.0,1921-06-01,84.936986,118671
4,ADNI1,002_S_0295X1a_118671,1673.5839558839798,3rd Ventricle,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",1.0,1.0,1921-06-01,84.936986,118671


In [37]:
full_merged['complete_diagnosis'].unique()
# result: 
# array(['1.0', '3.0', '2.0', '5.0', '7.0', '4.0', '6.0', '8.0', 'nan'],
# dtype=object)

# Code Details 
# 1=Stable: NL to NL; 2=Stable: MCI to MCI; 
# 3=Stable: Dementia to Dementia; 4=Conversion: NL to MCI; 
# 5=Conversion: MCI to Dementia; 6=Conversion: NL to Dementia; 
# 7=Reversion: MCI to NL; 8=Reversion: Dementia to MCI; 
# 9=Reversion: Dementia to NL

# should remove the Reversion codes*

array(['1.0', '3.0', '2.0', '5.0', '7.0', '4.0', '6.0', '8.0', 'nan'],
      dtype=object)

In [38]:
# Remove diagnoses of the following values:
#  

full_merged = (full_merged[(full_merged['complete_diagnosis'] != '7.0') 
                           & (full_merged['complete_diagnosis'] != '8.0') 
                           & (full_merged['complete_diagnosis'] != '9.0')
                           & (full_merged['complete_diagnosis'] != 'nan')])

In [39]:
full_merged['complete_diagnosis'].unique()

array(['1.0', '3.0', '2.0', '5.0', '4.0', '6.0'], dtype=object)

In [40]:
full_merged.head(5)

,project,code,roi_volume,roi_name,examDate,icv_values,%_icv,split_code,complete_diagnosis,PTGENDER,DOB_DATE,age_in_years,IMAGEUID
0,ADNI1,002_S_0295X1a_118671,1673.5839558839798,3rd Ventricle,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",1.0,1.0,1921-06-01,84.936986,118671
1,ADNI1,002_S_0295X1a_118671,1673.5839558839798,3rd Ventricle,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",1.0,1.0,1921-06-01,84.936986,118671
2,ADNI1,002_S_0295X1a_118671,1673.5839558839798,3rd Ventricle,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",1.0,1.0,1921-06-01,84.936986,118671
3,ADNI1,002_S_0295X1a_118671,1673.5839558839798,3rd Ventricle,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",1.0,1.0,1921-06-01,84.936986,118671
4,ADNI1,002_S_0295X1a_118671,1673.5839558839798,3rd Ventricle,2006-04-18,0.0,0.0009211534934467013,"['002', 'S', '0295X1a', '118671']",1.0,1.0,1921-06-01,84.936986,118671


In [41]:
#check match for .nii.gz images from ucsf box:
# https://ucsf.app.box.com/folder/154227125425?utm_campaign=collab%20auto%20accept%20user&utm_medium=email&utm_source=trans
# full_merged[full_merged['code']== '098_S_0160X7a_172295']
# for the above example: 612 rows

In [42]:
# check for redundant rows:
# full_merged.duplicated() #there are many :,) #

full_merged = full_merged.drop_duplicates()

In [43]:
full_merged[full_merged.duplicated()]

# yay, no more duplicates, , 




,project,code,roi_volume,roi_name,examDate,icv_values,%_icv,split_code,complete_diagnosis,PTGENDER,DOB_DATE,age_in_years,IMAGEUID


# Export dataset! :,)

In [44]:
full_merged.to_csv(r'C:/Users/glh19/OneDrive/Desktop/Winter_Spring 2022/Tosun-Turgut/data/output/adni_age_data_diag_clean.csv', index=False)
